# Legalis BERT Predictor

#### Notebook used to create BERT transformer model for text classification based prediction of outcomes

## importing packages and dataset
#### install via pip (needed especially for only jupyter notebook services)

In [ ]:
import huggingface_hub
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install huggingface_hub
!pip install accelerate -U
!pip install transformers[torch]
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.

#### imports for model fune tuning and dataset loading

In [1]:
# standard imports
import numpy as np
import os

#imports from huggingface
import datasets as ds #for loading datasets
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding # for text classification
import huggingface_hub as hg_hub #for model upload with hub
import evaluate #for accuracy evaluation

#### login to huggingface hub for model upload (only needed for model upload)

In [ ]:
#login to huggingface hub
hg_hub.login(os.environ['hub_token'])

#### loading dataset from huggingface hub or via csv

In [2]:
#loading dataset from csv (reproduction way)
#dataset=ds.load_dataset("csv", data_files="legalis-dataset.csv", split="train")
#dataset=dataset.train_test_split(test_size=0.2, shuffle=True)

#loading dataset from huggingface hub (production way)
dataset=ds.load_dataset("LennardZuendorf/legalis")

#renaming columns for easier access
dataset=dataset.rename_column("facts", "text")

#printing dataset features
print(dataset)

Found cached dataset csv (C:/Users/lenna/.cache/huggingface/datasets/csv/default-9d276a5be705aa8e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


DatasetDict({
    train: Dataset({
        features: ['id', 'file_number', 'date', 'type', 'content', 'tenor', 'facts', 'reasoning', 'winner', 'label'],
        num_rows: 2240
    })
    test: Dataset({
        features: ['id', 'file_number', 'date', 'type', 'content', 'tenor', 'facts', 'reasoning', 'winner', 'label'],
        num_rows: 561
    })
})


## data preprocessing & eval prep
#### tokenizing data with tokenizer used by German BERT model

In [ ]:
#grabbing tokenizer from pretrained german BERT
tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-base")

#creating function to tokenize data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

#tokenizing data
tokenized_data = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/2660 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

#### creating function to compute metrics using huggingface evaluations

In [ ]:
#creating accuracy evaluation class
accuracy = evaluate.load("accuracy")

#creating function to compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

#### transformers.trainer setup for training with BERT

In [ ]:
#setting up labels
id2label = {0: "Verklagte*r", 1: "Klaeger*in"}
label2id = {"Verklagte*r": 0, "Klaeger*in": 1}

#setting up model from huggingface with labels
model = AutoModelForSequenceClassification.from_pretrained(
    "deepset/gbert-base", num_labels=2, id2label=id2label, label2id=label2id
)

#training arguments and trainer setup
training_args = TrainingArguments(
    output_dir="legalis-BERT",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

#trainer setup with model, training args, datasets, tokenizer, data collator and compute metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly

#### training using the transformers trainer

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.667719,0.581560
2,No log,0.666185,0.602837


TrainOutput(global_step=334, training_loss=0.6516124245649326, metrics={'train_runtime': 155.6689, 'train_samples_per_second': 34.175, 'train_steps_per_second': 2.146, 'total_flos': 1399750814515200.0, 'train_loss': 0.6516124245649326, 'epoch': 2.0})

#### pushing model to huggingface hub (only needed for model upload)

In [ ]:
trainer.push_to_hub("legalis-BERT")